# Classification with Machine Learning classifiers and Feature Descriptors

In this lesson, we learn how to solve a classification problem through Machine Learning classifiers and two different Feature Descriptors.

**It is absolutely recommended to read the documentation relating to the functions and methods used!**
Usually, it is sufficient typing on Google the name of the function (and eventually the name of the library used).

Let's start importing some libraries.
In particular, `sklearn` is the library for the **Machine Learning stuff**!

In [ ]:
import sklearn
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from glob import glob
from os.path import join
import cv2
from skimage.feature import hog, local_binary_pattern
from tqdm import tqdm


The seed is important to have **deterministic** experiments.

In [ ]:
np.random.seed(1821)

# Functions and Classes
`get_labels()` is a function that receives a name (`string`) and returns the class (`int`), following this:

*   Triangle: 0
*   Rectangle: 1
*   Square: 2
*   Rhombus: 3

Example: 0_triangle.png → 0

In [ ]:
def get_labels(name):
    if 'triangle' in name:
        return 0
    elif 'square' in name:
        return 1
    elif 'rectangle' in name:
        return 2
    elif 'rhombus' in name:
        return 3
    else:
        raise NotImplementedError('Not existing class!')

`extract_feature()` is a function that, given a list of images, compute a Feature Descriptor.
We are going to use two libraries: `opencv` to handle images (opening, resizing) and `skimage` to compute features.

Specifically:
* `feat_type=1` → function computes HOG
* `feat_type=2` → function computes LBP
* `feat_type=3` → function does not computes any feature, but simply unroll the input image

**Tools**:
*   `cv2.imread()`: open an image (0: gray level, 1: BGR)
*   `cv2.resize()`: resize an image
*   `hog()`: compute HOG feature
*   `lbp()`: compute LBP feature

Be aware that the feature computation time (and the final accuracy of the model) are strictly related to the image size!


In [ ]:
def extract_features(images, feat_type, img_size):

    labels = []
    features = []

    for image in tqdm(images):

        img = cv2.imread(image, 0)

        img = cv2.resize(img, (img_size, img_size))

        if feat_type == 'hog':
            feat = hog(img, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1))
        elif feat_type == 'lbp':
            feat = np.ravel(local_binary_pattern(img, P=100, R=5))
        elif feat_type == 'img':
            img = img / 256.0
            feat = np.ravel(img)
        else:
            raise NotImplementedError('Not implemented feature!')

        features.append(feat)
        labels.append(get_labels(image))

    return features, labels

### Data


1.   Upload the `.zip` file containing the *Euclid* dataset
2.   Unzip the file using the following comand. Dataset folders will appear in `/content`



In [ ]:
!unzip -q Euclid_dataset.zip -d /content

Let's create a list of all images available in the dataset.

**Tools**:
* `join()`: joins one or more path components intelligently. The return value is the concatenation of path.
* `glob()`: returns a possibly empty list of path names that match names. Wildcards (in that case `*` are allowed!)

In [ ]:
dataset_path = '/content/Euclid_dataset'
images = glob(join(dataset_path, '*', '*.png'))
print('Images: ', len(images))

in this case it is essential to have a **training**, **validation** and **test** sets.

Training data are used to train the model, while the validation split is used to assess performance.

Here, we use validation and test set as synonymous, since we do not have a real test set.

We put **60% of data in training**, **10% in validation**, and the remaining **30% in the test set**.

In [ ]:
np.random.shuffle(images)
trainset = images[:int(0.6*len(images))]
valset = images[int(0.6*len(images)):int(0.7*len(images))]
testset = images[int(0.7*len(images)):]
print('Total: {} splitted in Train: {}, Val: {} and Test: {}'.format(len(images), len(trainset), len(valset), len(testset)))

Here, we define two important elements: the **size of the images** (used to compute feature descriptors) and the **type of features**.
We use a **progress bar** (`tqdm`) to show the state of the feature computation!

In [ ]:
img_size = 112
feature_type = 'hog'

train_x, train_y = extract_features(trainset, feature_type, img_size)
val_x, val_y = extract_features(valset, feature_type, img_size)
test_x, test_y = extract_features(testset, feature_type, img_size)

### Classifier
Now that the feature extraction is ended, we can define our classifiers.
As in the previous case, we start our analysis from the SVM.

In [ ]:
clf = svm.SVC(gamma=0.001, C=100., kernel='rbf', verbose=False)

### Training
Now we are ready for the training!
With `sklearn` library is tremendously simple, we just need training data (`train_x` and the related labels `train_y`) and pass them to the classifier.

**Tools**:
-   `model.fit()`: fit the provided model with training data.

In [ ]:
clf.fit(train_x, train_y)

### Validation

In [ ]:
clf.score(val_x, val_y)

### Testing
Now we are reading to use our classifier! The trained classifier output the labels (as defined above) for the classification task.

Tools:
  - `model.predict()`: predict the class of the given data.

In [ ]:
y_pred = clf.predict(test_x)
print('Predicted {} samples: {}'.format(len(y_pred), y_pred))

It's time to understand the final performance of the trained classifier.

**Tools**:
   * `accuracy_score()`: Accuracy classification score. The set of labels predicted for a sample must exactly match the corresponding set of labels of GT.

In [ ]:
print('Final Accuracy: {:.3f}'.format(accuracy_score(test_y, y_pred)))

### Confusion matrix

We can also compute the confusion matrix to further understand the performance on the trained model.

**Tools**:
   * `confusion_matrix()`: computes confusion matrix to evaluate the accuracy of a classification.
   * `plot_confusion_matrix()`: plots Confusion Matrix (it is deprectaed and will be removed in future versions of the library).

In [ ]:
matrix = confusion_matrix(test_y, y_pred)
print(matrix)
print(matrix.diagonal() / matrix.sum(axis=1))

cm = ConfusionMatrixDisplay(matrix)
cm.plot()

### Exercise

Try to use different **classifiers**.
In addition, you can try to change the parameters used to resize images and compute features:
- What changes can you note in **computational load**?
- What changes can you note in **performance**?

What further considerations can you observe thanks to the confusion matrix?


### Homework
1) Write your own function to compute the **confusion matrix** and the **diagonal** with the classification scores for each class.

In [ ]:
def compute_confusion_matrix(test_y, pred_y):

  # compute the confusion matrix

  # compute the diagonal

  pass

In [ ]:
compute_confusion_matrix(test_y, pred_y)

2) Download the CIFAR 10 dataset from Virtuale (`CIFAR-10-simple.zip`).
Load the dataset and the classes, try to solve the classification problem. You can use only the data provided in the `train` folder to train your model.
Compute the final accuracy with the folder `test`.

CIFAR 10 (https://www.cs.toronto.edu/~kriz/cifar.html) consists in **32x32 colour images** (RGB) divided in **10 classes**. There are **300 samples in training** and **50 testing samples** for each class.
